# 有限体の3次拡大体から有限射影平面を拵え巡回平面から巡回完全差集合と魔円陣を得る
### プログラミング言語は **Julia** を用いるが, **Nemo** のような代数的な **pkg** は使わず, 配列と行列と剰余計算のみで実装する
先ずは実際に実行している環境を確認し, 必要な **Pkg** を導入しておく

In [1]:
println(versioninfo())
using Combinatorics,Dates,Primes

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M3
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
nothing


### 先ずは$p$を素数として, 位数$p$の有限体$GF(p)$から拡大体$GF(p^n)$を構成することから
その為には, 先ず $GF(p)$上の(の元を係数とする)$n$次の多項式を想定して, それが既約多項式かどうかをチェックする必要がある。  
n次の既約多項式は, その係数の配列($\textsf{cl}$)として扱うが, 最高次の係数は 1 でこれを係数の配列には含めない。  
したがって, 例えば $3$ 次の多項式で $x^3+2x^2+1$ の場合, $1$ は含めず, $\textsf{cl}=[2,0,1]$ となる。  
その長さは $4$ ではなくて $3$ である。
#### 先ずは既約多項式を1つランダムに得ることから

In [2]:
# -------------------------------------------------------------------------
all_perm(x, n) = vec(map(collect, Iterators.product(ntuple(_ -> x, n)...)))# 配列 x の要素を n 個並べた「重複順列」の配列を作る
# -------------------------------------------------------------------------
function IrreduciblePolynomialChecker(p,cl)# GF(p)の n 次の既約多項式かどうかをチェックする p:位数, cl:係数の配列, 但し最高次の 1 は既定とし含めず
    n=length(cl)# 最高次数 
    GFp=Vector(0:p-1)# 整数    GFp=Vector(0:p-1)# 整数で処理するための, GF(p) は整数の配列で設定
    ans=true
    for x ∈ GFp# GFpの元を代入して, 0になれば「駄目」を返す
        val=x^n# 最高次の係数は 1 なので, cl(係数の配列)を見る事なく計算できる
        for i=1:n# 以下次数毎に代入して追加していく 但し, GF(p)上なので p の剰余計算を施しておく（最後に一括でも良いのだけど）
            val=(val+cl[i]*x^(n-i))%p
        end
        if val==0
            ans=false
            break
        end
    end
    ans
end
function IrreduciblePolynomialPicker(p,n)# GF(p)をGF(p^n)に拡大する際に利用する既約多項式を一つランダムに返す
    GFp=Vector(0:p-1)# 整数    GFp=Vector(0:p-1)# 整数で処理するための, GF(p) は整数の配列で設定
    chk=false
    cl=[]
    while !chk
        cl=[]
        for i=1:n
            push!(cl,rand(0:p-1))
        end
        if IrreduciblePolynomialChecker(p,cl)
            chk=true
            break
        end
    end
    cl
end
# -------------------------------------------------------------------------
p,n=3,4
cl=IrreduciblePolynomialPicker(p,n)
println(cl)

Any[0, 0, 0, 1]


### 次に原始多項式かどうかのチェック機能を追加して, 原始多項式を1つ得る事を実現する
原始多項式かどうかは, 既約多項式の根を $\alpha$ として, その累乗 $\alpha^i$ が $0$ を除く $GF(p^n)$ の元を構成するかどうかで確認できる。  
構成するかどうかは, 結局のところ, $\alpha^i$ の数重複なしに $p^n-1$ 個できることで確認できる。  
このとき, $\alpha$ は $n$ 次の既約多項式の根だから, $\alpha^n$ は n次以下の $\alpha$ の多項式で表される。  

In [19]:
# -------------------------------------------------------------------------
function makeGFpnElementList(p,cl)# GF(p) を 係数配列 cl な原始多項式(但し, 最高次な n 次の係数 1 は配列外) で作成した 
#                                   拡大体 GF(p^n) の元を, 配列の配列で列挙する。
#                                   n は原始多項式 cl の係数配列の「大きさ/長さ」になっているので指定は不要な点に注意
    n=length(cl)# GF(p^n)の n は既約多項式の係数の配列の長さから分かる
    Alp=zeros(Int64,n,n)# ゼロ行列を用意して, 以下 「αの累乗」を計算する為の行列の準備をする。
    for i=2:n
        Alp[i-1,i]=1
    end
    for i=1:n
        Alp[i,1]=(2*p-cl[i])%p
    end
#    display(Alp)
    be=Vector(1:n).*0# ゼロ配列を準備する。GF(p^n)の元としては, これがゼロになる。
    ansl=[Vector(1:n).*0]# 関数が返す αの累乗を収めておく配列。最終これを返すこととなる。
    e=copy(be)
    e[n]=1# e=[0,0,...,0,1]とする。GF(p^n)の元としては, これが１になる。
    push!(ansl,e)# 関数の返すαの累乗を収める配列anslに1も入れておく。
#    println(e)
    chk=true
    for i=1:p^n-1
        e=(Alp*e).%p
#        println(e)
        if e ∈ ansl
            chk=false
#            println(" *** Break ***")
            break
        else
            push!(ansl,e)
        end
    end
    unique(ansl)
end
function PrimitivePolynomialPicker(p,n)
    chk=false
    while !chk
        cl=IrreduciblePolynomialPicker(p,n)
        Fpnl=unique!(makeGFpnElementList(p,cl))
#        println(cl,",",Fpnl)
        if length(Fpnl)==p^n
            return cl
            chk=true
            break
        end
    end
end
# -------------------------------------------------------------------------
p,n=3,2
cl=PrimitivePolynomialPicker(p,n)
Fpnl=makeGFpnElementList(p,cl)
# -------------------------------------------------------------------------
function SimpleAryAry(aryl)
    prnstrl=[]# 以下で得られた GF(p^n)の元を配列表現で一覧にしている。
    for e ∈ aryl
        push!(prnstrl,"["*join(e,",")*"]")
    end
    println(join(prnstrl,","))
end
SimpleAryAry(Fpnl)

[0,0],[0,1],[1,0],[2,1],[2,2],[0,2],[2,0],[1,2],[1,1]


### 行列は計算上どう役になっているのか
$GF(p^n)$ の元の配列を返す **makeGFpnElementList(p,cl)** の最初に行列 **Alp** が登場します。  
$n\times n$の正方行列で, この**Alp**に元を表す配列を掛けると, $\alpha$を掛ける計算ができるというわけです。  
そもそも $GF(p^n)$ の元は, 原始多項式の根である $\alpha$ の累乗で構成される, $\alpha$ の高々 $n-1$次の多項式です。  
そしてその「$\alpha$ の高々 $n-1$次の多項式」も係数で扱うことになります。  
例えば, $p=5$ として $2\alpha^4+3\alpha^3+4\alpha^2+\alpha+2$ だと $[2,3,4,1,2]$ でプログラム上では処理します。  
$1$ なら $[0,0,0,0,1]$ だし, $\alpha$ なら $[0,0,0,1,0]$ というわけです。  
そして, $\alpha^4$ だと $[1,0,0,0,0]$ となるわけで, $\alpha$の累乗の最初の方は, 配列がズレるだけで簡単です。  
では $\alpha^5$ はどうなるのかというと, ここで, 原始多項式が効いてきます。
仮に $x^5+2x^4+3x^3+4x^2+x+2$ が原始多項式だとすれば, $\alpha$ は $x^5+2x^4+3x^3+4x^2+x+2=0$ の解なので, 
$\alpha^5=-2\alpha^4-3\alpha^3-4\alpha^2-\alpha-2$となります。  
配列で確認すれば, $\alpha^4$ である $[1,0,0,0,0]$ に $\alpha$ を掛けると, $[-2,-3,-4,-1,-2]$ となるというわけです。  
ただ $-$ は使わないので, 剰余の計算で, $[3,2,1,4,3]$ となるということになりますね。  
纏えて書けば, 
$$
\begin{align}
&1\        な &\ [0,0,0,0,1] &\ に\alpha を掛けると, &\ \alpha\   になるので, &\ [0,0,0,1,0]\\
&\alpha\   な &\ [0,0,0,1,0] &\ に\alpha を掛けると, &\ \alpha^2\ になるので, &\ [0,0,1,0,0]\\
&\alpha^2\ な &\ [0,0,1,0,0] &\ に\alpha を掛けると, &\ \alpha^3\ になるので, &\ [0,1,0,0,0]\\
&\alpha^3\ な &\ [0,1,0,0,0] &\ に\alpha を掛けると, &\ \alpha^4\ になるので, &\ [1,0,0,0,0]\\
&\alpha^4\ な &\ [1,0,0,0,0] &\ に\alpha を掛けると, &\ \alpha^5\ になるので, &\ [3,2,1,4,3]
\end{align}
$$
ということです。次の行列 $Alp$ を用意しておけば,    
$$
Alp=
\left(\begin{array}{ccccc}
3 & 1 & 0 & 0 & 0 \\
2 & 0 & 1 & 0 & 0 \\
1 & 0 & 0 & 1 & 0 \\
4 & 0 & 0 & 0 & 1 \\
5 & 0 & 0 & 0 & 0 
\end{array}\right)
$$
$GF(p^n)$ の元である $\alpha^i=[a,b,c,d,e]$にこの $Alp$ を掛けることで, $\alpha^{i+1}$ の配列が計算できるというわけです。
（同じ様なことを, あとで3次拡大する時にも考えます。）

In [27]:
p,n=7,4
cl=PrimitivePolynomialPicker(p,n)
println(cl)
Fpnl=makeGFpnElementList(p,cl)
#SimpleAryAry(Fpnl)
println("Size of GF(p^n)=",length(Fpnl),", p^n=",p^n)

Any[0, 3, 4, 3]
Size of GF(p^n)=2401, p^n=2401


### 次にこの$GF(p^n)$を 3次拡大します。
同じようなことを考えるわけで, ただ今回は3次拡大と決まっていますから, 3次の原始多項式が設定できれば良いとなります。  
違うのは, 既約多項式や原始多項式の係数は, $GF(p^n)$の元であり, ここでは配列を使っています。  
どうすれば良いでしょう。  
多項式の候補は, **makeGFpnElementList(p,cl)**で列挙できる$GF(p^n)$の元から選べば良いでしょう。  
ただ既約多項式かどうかのチェックには, その配列係数の多項式に, 配列形式な$GF(p^n)$の元を代入するという事になります。  